In [39]:
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
import seaborn
import requests

Here I will be doing some webscraping with BeautifulSoup

In [40]:
url = 'https://en.wikipedia.org/wiki/List_of_Marvel_Cinematic_Universe_films'
request = requests.get(url)
request

<Response [200]>

In [ ]:
content = request.text
content

In [42]:
soup = BeautifulSoup(content)

In [ ]:
soup.find_all('table')

In [44]:
table = soup.find('table', class_ = 'wikitable plainrowheaders sortable') 

In [46]:
header = table.find_all('th')[0:11]
upper_header = []
lower_header = []
first_entry = []
for val in header:
    if 'colspan' in val.attrs: # if there is a colspan attribute, the header was in the upper level 
        upper_header.append(val.text.strip())
    elif 'rowspan' not in val.attrs: # If there is not a rowspan attribute, the header is on the lower level 
        lower_header.append(val.text.strip())
    else:
        first_entry.append(val.text.strip())
    
columns = [col.text.strip() for col in header]

first_entry.pop() # The wikipedia table was set up to have multiple rows in the header, so I need to break it down into just the headers I want
last_entry = first_entry.pop()
upper_header.pop()
lower_header.pop()
lower_header.pop()
second_entry = []
for entry in lower_header:
    second_entry.append(upper_header[0] + '-' + entry)


column_names = first_entry + second_entry + [last_entry]
column_names

['Film',
 'U.S. release date',
 'Box office gross-U.S. and Canada',
 'Box office gross-Other territories',
 'Box office gross-Worldwide',
 'Budget']

In [47]:
all_row_data = []
names = []
rows = table.find_all('tr')[4:]
for row in rows:
    name = row.find_all('th')
    name_info = [val.text.strip() for val in name]
    tmp1 = name_info
    row_data = row.find_all('td')
    info = [data.text.strip() for data in row_data]
    tmp2 = info
    tmp1 = tmp1 + tmp2
    all_row_data.append(tmp1)

all_row_data.pop()
all_row_data
    

[['Iron Man',
  'May 2, 2008',
  '$319,034,126',
  '$266,762,121',
  '$585,796,247',
  '89',
  '190',
  '$140\xa0million',
  '[229]'],
 ['The Incredible Hulk',
  'June 13, 2008',
  '$134,806,913',
  '$129,964,083',
  '$264,770,996',
  '494',
  '625',
  '$137.5–150\xa0million',
  '[230][231]'],
 ['Iron Man 2',
  'May 7, 2010',
  '$312,433,331',
  '$311,500,000',
  '$623,933,331',
  '95',
  '172',
  '$170–200\xa0million',
  '[232][233]'],
 ['Thor',
  'May 6, 2011',
  '$181,030,624',
  '$268,295,994',
  '$449,326,618',
  '280',
  '290',
  '$150\xa0million',
  '[234]'],
 ['Captain America: The First Avenger',
  'July 22, 2011',
  '$176,654,505',
  '$193,915,269',
  '$370,569,774',
  '298',
  '394',
  '$140\xa0million',
  '[235]'],
 ["Marvel's The Avengers",
  'May 4, 2012',
  '$623,357,910',
  '$897,180,626',
  '$1,520,538,536',
  '12',
  '10',
  '$220\xa0million',
  '[236][237]'],
 ['Phase Two'],
 ['Iron Man 3',
  'May 3, 2013',
  '$409,013,994',
  '$806,563,211',
  '$1,215,577,205',
  '3

In [48]:
all_row_data = [list for list in all_row_data if len(list) != 0] # remove any empty entries 
all_row_data

[['Iron Man',
  'May 2, 2008',
  '$319,034,126',
  '$266,762,121',
  '$585,796,247',
  '89',
  '190',
  '$140\xa0million',
  '[229]'],
 ['The Incredible Hulk',
  'June 13, 2008',
  '$134,806,913',
  '$129,964,083',
  '$264,770,996',
  '494',
  '625',
  '$137.5–150\xa0million',
  '[230][231]'],
 ['Iron Man 2',
  'May 7, 2010',
  '$312,433,331',
  '$311,500,000',
  '$623,933,331',
  '95',
  '172',
  '$170–200\xa0million',
  '[232][233]'],
 ['Thor',
  'May 6, 2011',
  '$181,030,624',
  '$268,295,994',
  '$449,326,618',
  '280',
  '290',
  '$150\xa0million',
  '[234]'],
 ['Captain America: The First Avenger',
  'July 22, 2011',
  '$176,654,505',
  '$193,915,269',
  '$370,569,774',
  '298',
  '394',
  '$140\xa0million',
  '[235]'],
 ["Marvel's The Avengers",
  'May 4, 2012',
  '$623,357,910',
  '$897,180,626',
  '$1,520,538,536',
  '12',
  '10',
  '$220\xa0million',
  '[236][237]'],
 ['Phase Two'],
 ['Iron Man 3',
  'May 3, 2013',
  '$409,013,994',
  '$806,563,211',
  '$1,215,577,205',
  '3

In [49]:
row_data_copy = all_row_data

for element in row_data_copy:
    element.pop() # The last entry is for a reference column, which I did not use, so it will  be dropped 
    del element[5:7] # This slice also contains values from a column which i did not use, so it will be dropped 
    
row_data_copy

[['Iron Man',
  'May 2, 2008',
  '$319,034,126',
  '$266,762,121',
  '$585,796,247',
  '$140\xa0million'],
 ['The Incredible Hulk',
  'June 13, 2008',
  '$134,806,913',
  '$129,964,083',
  '$264,770,996',
  '$137.5–150\xa0million'],
 ['Iron Man 2',
  'May 7, 2010',
  '$312,433,331',
  '$311,500,000',
  '$623,933,331',
  '$170–200\xa0million'],
 ['Thor',
  'May 6, 2011',
  '$181,030,624',
  '$268,295,994',
  '$449,326,618',
  '$150\xa0million'],
 ['Captain America: The First Avenger',
  'July 22, 2011',
  '$176,654,505',
  '$193,915,269',
  '$370,569,774',
  '$140\xa0million'],
 ["Marvel's The Avengers",
  'May 4, 2012',
  '$623,357,910',
  '$897,180,626',
  '$1,520,538,536',
  '$220\xa0million'],
 [],
 ['Iron Man 3',
  'May 3, 2013',
  '$409,013,994',
  '$806,563,211',
  '$1,215,577,205',
  '$200\xa0million'],
 ['Thor: The Dark World',
  'November 8, 2013',
  '$206,362,140',
  '$438,421,000',
  '$644,783,140',
  '$150–170\xa0million'],
 ['Captain America: The Winter Soldier',
  'April 

In [50]:
df = pd.DataFrame(row_data_copy, columns = column_names)
df.dropna(inplace = True) # do not want na values 

In [51]:
def replace_millions(x): # Need to remove the weird million character and replace with zeros, but this will make some weird entries with decimal points that will be handled later 
    if '\xa0million' in x:
        return x.split('\xa0million')[0] + ',000,000'
    else:
        return x
    

In [52]:
col_change = df['Budget']
new_budget = col_change.apply(replace_millions)
new_budget

0             $140,000,000
1       $137.5–150,000,000
2         $170–200,000,000
3             $150,000,000
4             $140,000,000
5             $220,000,000
7             $200,000,000
8         $150–170,000,000
9         $170–177,000,000
10            $170,000,000
11        $250–444,000,000
12            $130,000,000
14            $250,000,000
15            $165,000,000
16            $200,000,000
17            $175,000,000
18            $180,000,000
19            $200,000,000
20        $325–400,000,000
21            $162,000,000
22        $150–175,000,000
23        $356–400,000,000
24            $160,000,000
26            $200,000,000
27        $150–200,000,000
28          $236.2,000,000
29            $200,000,000
30        $172–200,000,000
31            $250,000,000
32            $250,000,000
34      $276–326.6,000,000
35            $250,000,000
36    $219.8–274.8,000,000
37            $200,000,000
Name: Budget, dtype: object

In [53]:
def remove_hyphen(x): # Some of the budgets for films were given as a range, so i just took the maximum value in that range
    if '–' in x: # checks for range of values
        tmp = x.split('–')[1] # second part of the string 
        if '.' in tmp: # need to correct ranges with decimal points caused by prior data cleaning 
            tmp2 = tmp.split('.')[1] # Rebuilding the corrected string using values from various splits 
            mil_val = tmp.split('.')[0]
            thousands_val = tmp2.split(',')[0]
            zeros = tmp2.split(',')[1]
            hundred = tmp2.split(',')[2]
            zeros = thousands_val + zeros[:-1]
            return '$' + mil_val + ',' + zeros + ',' + hundred      
        else:
            return '$' + x.split('–')[1] # If there is no decimal point in the range then it is pretty simple to format 
    else:
        if '.' in x: # Some of the values have a decimal point without a range, so need to handle those as well 
            tmp2 = x.split('.')[1]
            mil_val = x.split('.')[0]
            thousands_val = tmp2.split(',')[0]
            zeros = tmp2.split(',')[1]
            hundred = tmp2.split(',')[2]
            zeros = thousands_val + zeros[:-1]
            return mil_val + ',' + zeros + ',' + hundred   
        return x


In [54]:
final_budget = new_budget.apply(remove_hyphen)
final_budget

0     $140,000,000
1     $150,000,000
2     $200,000,000
3     $150,000,000
4     $140,000,000
5     $220,000,000
7     $200,000,000
8     $170,000,000
9     $177,000,000
10    $170,000,000
11    $444,000,000
12    $130,000,000
14    $250,000,000
15    $165,000,000
16    $200,000,000
17    $175,000,000
18    $180,000,000
19    $200,000,000
20    $400,000,000
21    $162,000,000
22    $175,000,000
23    $400,000,000
24    $160,000,000
26    $200,000,000
27    $200,000,000
28    $236,200,000
29    $200,000,000
30    $200,000,000
31    $250,000,000
32    $250,000,000
34    $326,600,000
35    $250,000,000
36    $274,800,000
37    $200,000,000
Name: Budget, dtype: object

In [55]:
final_budget[28]

'$236,200,000'

In [56]:
df['Budget'] = final_budget

In [57]:
df.loc[26, 'Box office gross-Worldwide'] = '379,751,655' # For some reason this was not being stored correctly from previous steps, so i manually entered it 

In [58]:
df

,Film,U.S. release date,Box office gross-U.S. and Canada,Box office gross-Other territories,Box office gross-Worldwide,Budget
0,Iron Man,"May 2, 2008","$319,034,126","$266,762,121","$585,796,247","$140,000,000"
1,The Incredible Hulk,"June 13, 2008","$134,806,913","$129,964,083","$264,770,996","$150,000,000"
2,Iron Man 2,"May 7, 2010","$312,433,331","$311,500,000","$623,933,331","$200,000,000"
3,Thor,"May 6, 2011","$181,030,624","$268,295,994","$449,326,618","$150,000,000"
4,Captain America: The First Avenger,"July 22, 2011","$176,654,505","$193,915,269","$370,569,774","$140,000,000"
5,Marvel's The Avengers,"May 4, 2012","$623,357,910","$897,180,626","$1,520,538,536","$220,000,000"
7,Iron Man 3,"May 3, 2013","$409,013,994","$806,563,211","$1,215,577,205","$200,000,000"
8,Thor: The Dark World,"November 8, 2013","$206,362,140","$438,421,000","$644,783,140","$170,000,000"
9,Captain America: The Winter Soldier,"April 4, 2014","$259,766,572","$454,654,931","$714,421,503","$177,000,000"
10,Guardians of the Galaxy,"August 1, 2014","$333,718,600","$439,631,547","$773,350,147","$170,000,000"


In [59]:
def convert_to_int(x): # Need to convert each of the strings into integers for future analysis 
    return int(x.replace(',', '')[1:]) # Remove comma, ignore first character of string ($)

In [60]:
worldwide = df['Box office gross-Worldwide']
new_ww = worldwide.apply(convert_to_int)
new_ww

0      585796247
1      264770996
2      623933331
3      449326618
4      370569774
5     1520538536
7     1215577205
8      644783140
9      714421503
10     773350147
11    1405018048
12     519311965
14    1155046416
15     677796076
16     863756051
17     880166924
18     855301806
19    1349926083
20    2052415039
21     622674139
22    1131416446
23    2799439100
24    1131927996
26      79751655
27     432243292
28     402064899
29    1921847111
30     955775804
31     760928081
32     859208836
34     476071180
35     845555777
36     206136557
37    1161636400
Name: Box office gross-Worldwide, dtype: int64

In [61]:
others = df['Box office gross-Other territories']
new_others = others.apply(convert_to_int)


In [62]:
us = df['Box office gross-U.S. and Canada']
new_us = us.apply(convert_to_int)

In [63]:
budg = df['Budget']
new_budg = budg.apply(convert_to_int)

In [64]:
updated_df = df.copy()

In [65]:
updated_df['Box office gross-U.S. and Canada'] = new_us
updated_df['Box office gross-Other territories'] = new_others
updated_df['Box office gross-Worldwide'] = new_ww
updated_df['Budget'] = new_budg

In [66]:
updated_df

,Film,U.S. release date,Box office gross-U.S. and Canada,Box office gross-Other territories,Box office gross-Worldwide,Budget
0,Iron Man,"May 2, 2008",319034126,266762121,585796247,140000000
1,The Incredible Hulk,"June 13, 2008",134806913,129964083,264770996,150000000
2,Iron Man 2,"May 7, 2010",312433331,311500000,623933331,200000000
3,Thor,"May 6, 2011",181030624,268295994,449326618,150000000
4,Captain America: The First Avenger,"July 22, 2011",176654505,193915269,370569774,140000000
5,Marvel's The Avengers,"May 4, 2012",623357910,897180626,1520538536,220000000
7,Iron Man 3,"May 3, 2013",409013994,806563211,1215577205,200000000
8,Thor: The Dark World,"November 8, 2013",206362140,438421000,644783140,170000000
9,Captain America: The Winter Soldier,"April 4, 2014",259766572,454654931,714421503,177000000
10,Guardians of the Galaxy,"August 1, 2014",333718600,439631547,773350147,170000000


In [67]:
updated_df.to_csv('~/Documents/Marvel_Movie_Analysis/Marvel_Movie_Economics.csv', index = False)